In [1]:
%%bash 
which python

/global/project/projectdirs/atlas/xju/miniconda3/envs/py3.6/bin/python


In [2]:
import numpy as np
import pandas as pd
import time
import sklearn.metrics

from trackml.dataset import load_event

import networkx as nx

In [3]:
import sys
sys.path.append('..')

from datasets.graph import load_graph

## Prepare network-x graphs using hitsgraph

In [4]:
base_dir = '/global/cscratch1/sd/xju/heptrkx/data/hitgraphs_001/event00000{}_g{:03d}.npz'

In [5]:
def get_edge_features(in_node, out_node):
    # input are the features of incoming and outgoing nodes
    # they are ordered as [r, phi, z]
    in_r, in_phi, _   = in_node
    out_r, out_phi, _ = out_node
    in_x = in_r * np.cos(in_phi)
    in_y = in_r * np.sin(in_phi)
    out_x = out_r * np.cos(out_phi)
    out_y = out_r * np.sin(out_phi)
    return np.sqrt((in_x - out_x)**2 + (in_y - out_y)**2)

In [6]:
def networkx_graph_from_hitsgraph(ievt, isec):
    ## ievt start from 1000
    file_name = base_dir.format(ievt, isec)
    return hitsgraph_to_networkx_graph(load_graph(file_name))


def hitsgraph_to_networkx_graph(G):
    n_nodes, n_edges = G.Ri.shape
    
    graph = nx.DiGraph()

    ## add nodes
    for i in range(n_nodes):
        graph.add_node(i, pos=G.X[i], solution=0.0)
    
    for iedge in range(n_edges):
        in_node_id  = G.Ri[:, iedge].nonzero()[0][0]
        out_node_id = G.Ro[:, iedge].nonzero()[0][0]

        # distance as features
        in_node_features  = G.X[in_node_id]
        out_node_features = G.X[out_node_id]
        distance = get_edge_features(in_node_features, out_node_features)
        # add edges, bi-directions
        graph.add_edge(in_node_id, out_node_id, distance=distance, solution=G.y[iedge])
        graph.add_edge(out_node_id, in_node_id, distance=distance, solution=G.y[iedge])
        # add "solution" to nodes
        graph.node[in_node_id].update(solution=G.y[iedge])
        graph.node[out_node_id].update(solution=G.y[iedge])
        
        
    # add global features, not used for now
    graph.graph['features'] = np.array([0.])
    
    return graph

def graph_to_input_target(graph):
    def create_feature(attr, fields):
        return np.hstack([np.array(attr[field], dtype=float) for field in fields])
    
    input_node_fields = ("pos",)
    input_edge_fields = ("distance",)
    target_node_fields = ("solution",)
    target_edge_fields = ("solution",)
    
    input_graph = graph.copy()
    target_graph = graph.copy()
    
    for node_index, node_feature in graph.nodes(data=True):
        input_graph.add_node(
            node_index, features=create_feature(node_feature, input_node_fields)
        )
        target_graph.add_node(
            node_index, features=create_feature(node_feature, target_node_fields)
        )
        
    for receiver, sender, features in graph.edges(data=True):
        input_graph.add_edge(
            sender, receiver, features=create_feature(features, input_edge_fields)
        )
        target_graph.add_edge(
            sender, receiver, features=create_feature(features, target_edge_fields)
        )
        
    input_graph.graph['features'] = input_graph.graph['features'] = np.array([0.0])
    return input_graph, target_graph


def generate_input_target(n_graphs, start_evt_id=1000):
    input_graphs = []
    target_graphs = []
    for i in range(n_graphs):
        evt_id = start_evt_id + i
        isec = 0
        graph = networkx_graph_from_hitsgraph(evt_id, isec)
        input_graph, output_graph = graph_to_input_target(graph)
        input_graphs.append(input_graph)
        target_graphs.append(output_graph)
    return input_graphs, target_graphs


## Use interaction network from modules

In [7]:
import tensorflow as tf

In [8]:
from graph_nets import modules
from graph_nets import utils_tf
from graph_nets import utils_np

import sonnet as snt

In [9]:
tf.reset_default_graph()

### Define GNN model

In [10]:
NUM_LAYERS = 2  # Hard-code number of layers in the edge/node/global models.
LATENT_SIZE = 16  # Hard-code latent layer sizes for demos.


def make_mlp_model():
  """Instantiates a new MLP, followed by LayerNorm.

  The parameters of each new MLP are not shared with others generated by
  this function.

  Returns:
    A Sonnet module which contains the MLP and LayerNorm.
  """
  return snt.Sequential([
      snt.nets.MLP([LATENT_SIZE] * NUM_LAYERS, activate_final=True),
      snt.LayerNorm()
  ])

class MLPGraphIndependent(snt.AbstractModule):
  """GraphIndependent with MLP edge, node, and global models."""

  def __init__(self, name="MLPGraphIndependent"):
    super(MLPGraphIndependent, self).__init__(name=name)
    with self._enter_variable_scope():
      self._network = modules.GraphIndependent(
          edge_model_fn=make_mlp_model,
          node_model_fn=make_mlp_model,
          global_model_fn=None)

  def _build(self, inputs):
    return self._network(inputs)



class SegmentClassifier(snt.AbstractModule):

  def __init__(self, name="SegmentClassifier"):
    super(SegmentClassifier, self).__init__(name=name)

    self._encoder = MLPGraphIndependent()
    self._core = modules.InteractionNetwork(
        edge_model_fn=make_mlp_model,
        node_model_fn=make_mlp_model,
        reducer=tf.unsorted_segment_prod
    )

    # Transforms the outputs into the appropriate shapes.
    edge_output_size = 1
#     edge_fn = lambda: snt.Linear(edge_output_size, name="edge_output")
    edge_fn =lambda: snt.nets.MLP([edge_output_size], activation=tf.nn.tanh, name='edge_output')

    with self._enter_variable_scope():
      self._output_transform = modules.GraphIndependent(edge_fn, None, None)

  def _build(self, input_op, num_processing_steps):
    latent = self._encoder(input_op)

    output_ops = []
    for _ in range(num_processing_steps):
        core_input = latent
        latent = self._core(core_input)
        output_ops.append(self._output_transform(latent))
    return output_ops

In [11]:

model = SegmentClassifier()

### Write Loss functions and Feed-dict

In [12]:
global event_track
event_track = 1000
def create_feed_dict(batch_size, input_ph, target_ph):
    global event_track
    inputs, targets = generate_input_target(batch_size, event_track)
    event_track += batch_size

    input_graphs = utils_np.networkxs_to_graphs_tuple(inputs)
    target_graphs = utils_np.networkxs_to_graphs_tuple(targets)
    feed_dict = {input_ph: input_graphs, target_ph: target_graphs}
#         print(event_track)
    return feed_dict

In [13]:
def create_loss_ops(target_op, output_ops):
    # only use edges
    loss_ops = [
        tf.losses.sigmoid_cross_entropy(target_op.edges, output_op.edges)
        for output_op in output_ops
    ]
    return loss_ops


def make_all_runnable_in_session(*args):
  """Lets an iterable of TF graphs be output from a session as NP graphs."""
  return [utils_tf.make_runnable_in_session(a) for a in args]

In [ ]:
def computer_matrics(target, output):
    tdds = utils_np.graphs_tuple_to_data_dicts(target)
    odds = utils_np.graphs_tuple_to_data_dicts(output)
    
    test_target = []
    test_pred = []
    for td, od in zip(tdds, odds):
        test_target.append(td['edges'])
        test_pred.append(od['edges'])
    
    test_target = np.concatenate(test_target, axis=0)
    test_pred   = np.concatenate(test_pred,   axis=0)
    
    thresh = 0.5
    y_pred, y_true = (test_pred > thresh), (test_target > thresh)
    return sklearn.metrics.precision_score(y_true, y_pred), sklearn.metrics.recall_score(y_true, y_pred)

In [15]:
batch_size = n_graphs = 2
num_training_iterations = 10000
num_processing_steps_tr = 4  ## level of message-passing

In [16]:
input_graphs, target_graphs = generate_input_target(n_graphs)
input_ph  = utils_tf.placeholders_from_networkxs(input_graphs, force_dynamic_num_graphs=True)
target_ph = utils_tf.placeholders_from_networkxs(target_graphs, force_dynamic_num_graphs=True)

In [17]:
output_ops_tr = model(input_ph, num_processing_steps_tr)

# Training loss.
loss_ops_tr = create_loss_ops(target_ph, output_ops_tr)
# Loss across processing steps.
loss_op_tr = sum(loss_ops_tr) / num_processing_steps_tr

# Optimizer
learning_rate = 1e-3
optimizer = tf.train.AdamOptimizer(learning_rate)
step_op = optimizer.minimize(loss_op_tr)

# Lets an iterable of TF graphs be output from a session as NP graphs.
input_ph, target_ph = make_all_runnable_in_session(input_ph, target_ph)

/global/project/projectdirs/atlas/xju/miniconda3/envs/py3.6/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [ ]:
#@title Reset session  { form-width: "30%" }

# This cell resets the Tensorflow session, but keeps the same computational
# graph.

try:
  sess.close()
except NameError:
  pass
sess = tf.Session()
sess.run(tf.global_variables_initializer())

last_iteration = 0
logged_iterations = []
losses_tr = []
corrects_tr = []
solveds_tr = []


#@title Run training  { form-width: "30%" }

# You can interrupt this cell's training loop at any time, and visualize the
# intermediate results by running the next cell (below). You can then resume
# training by simply executing this cell again.

# How much time between logging and printing the current results.
log_every_seconds = 20

print("# (iteration number), T (elapsed seconds), "
      "Ltr (training loss), "
      "Precision, "
      "Recall")

start_time = time.time()
last_log_time = start_time
for iteration in range(last_iteration, num_training_iterations):
  last_iteration = iteration
  feed_dict = create_feed_dict(batch_size, input_ph, target_ph)
  train_values = sess.run({
      "step": step_op,
      "target": target_ph,
      "loss": loss_op_tr,
      "outputs": output_ops_tr
  }, feed_dict=feed_dict)
  the_time = time.time()
  elapsed_since_last_log = the_time - last_log_time

  if elapsed_since_last_log > log_every_seconds:
    last_log_time = the_time
    feed_dict = create_feed_dict(batch_size, input_ph, target_ph)
    test_values = sess.run({
        "target": target_ph,
        "loss": loss_op_tr,
        "outputs": output_ops_tr
    }, feed_dict=feed_dict)
    correct_tr, solved_tr = computer_matrics(
        test_values["target"], test_values["outputs"][-1])
    elapsed = time.time() - start_time
    losses_tr.append(train_values["loss"])
    corrects_tr.append(correct_tr)
    solveds_tr.append(solved_tr)
    logged_iterations.append(iteration)
    print("# {:05d}, T {:.1f}, Ltr {:.4f}, Lge {:.4f}, Precision {:.4f}, Recall"
          " {:.4f}".format(
              iteration, elapsed, train_values["loss"], test_values["loss"],
              correct_tr, solved_tr))

# (iteration number), T (elapsed seconds), Ltr (training loss), Precision, Recall
# 00013, T 22.8, Ltr 0.6372, Lge 0.6301, Precision 0.6904, Recall 0.0293
# 00027, T 43.2, Ltr 0.5904, Lge 0.6244, Precision 0.8113, Recall 0.0616
# 00042, T 65.1, Ltr 0.5759, Lge 0.5341, Precision 0.7420, Recall 0.0686
# 00057, T 86.2, Ltr 0.5313, Lge 0.5171, Precision 0.7730, Recall 0.1054
# 00072, T 105.6, Ltr 0.5454, Lge 0.5505, Precision 0.7383, Recall 0.2142
# 00086, T 126.4, Ltr 0.5515, Lge 0.4892, Precision 0.7317, Recall 0.1829
# 00099, T 146.5, Ltr 0.5668, Lge 0.5344, Precision 0.7743, Recall 0.2098
# 00113, T 166.9, Ltr 0.5395, Lge 0.5237, Precision 0.7402, Recall 0.2023
# 00127, T 187.4, Ltr 0.4933, Lge 0.5168, Precision 0.7376, Recall 0.1599
# 00140, T 207.3, Ltr 0.5417, Lge 0.5197, Precision 0.7889, Recall 0.2034
# 00154, T 228.0, Ltr 0.5143, Lge 0.4965, Precision 0.8231, Recall 0.1819
# 00167, T 248.5, Ltr 0.5001, Lge 0.5246, Precision 0.7828, Recall 0.2045
# 00182, T 271.1, Ltr 0.4934, Lge 

In [15]:



def compute_accuracy(target, output):
    tdds = utils_np.graphs_tuple_to_data_dicts(target)
    odds = utils_np.graphs_tuple_to_data_dicts(output)
    
    cs = []
    ss = []
    for td, od in zip(tdds, odds):
        xe = np.argmax(td['edges'], axis=-1)
        ye = np.argmax(od['edges'], axis=-1)
        c = []
        c.append(xe == ye)
        c = np.concatenate(c, axis=0)
        s = np.all(c)
        cs.append(c)
        ss.append(s)
    correct = np.mean(np.concatenate(cs, axis=0))
    solved = np.mean(np.stack(ss))
    return correct, solved

In [36]:
tf.reset_default_graph()

TypeError: output_sizes must be iterable

In [20]:
graph_tuple = utils_np.networkxs_to_graphs_tuple(input_graphs)
print(graph_tuple.receivers)
print(graph_tuple.nodes.shape)
print(graph_tuple.edges.shape)
print(graph_tuple.receivers[np.where(graph_tuple.receivers > graph_tuple.nodes.shape[0])])

[   1   86   92 ... 2402 2417 2632]
(2687, 3)
(16504, 1)
[]


# (iteration number), T (elapsed seconds), Ltr (training loss), Accuracy, Recall
# 00008, T 25.1, Ltr 0.5930, Lge 0.5660, Accuracy 0.3445, Recall 0.0086
# 00015, T 46.1, Ltr 0.5734, Lge 0.5504, Accuracy 0.2917, Recall 0.0017


/global/project/projectdirs/atlas/xju/miniconda3/envs/py3.6/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


# 00027, T 66.6, Ltr 0.5056, Lge 0.5433, Accuracy 0.0000, Recall 0.0000
# 00040, T 87.9, Ltr 0.5115, Lge 0.5213, Accuracy 1.0000, Recall 0.0035
# 00052, T 108.5, Ltr 0.5266, Lge 0.5079, Accuracy 0.9510, Recall 0.0672
# 00065, T 129.5, Ltr 0.5020, Lge 0.4900, Accuracy 0.9028, Recall 0.1077
# 00078, T 149.6, Ltr 0.5047, Lge 0.5065, Accuracy 0.8380, Recall 0.2242
# 00091, T 170.4, Ltr 0.4696, Lge 0.5240, Accuracy 0.8215, Recall 0.2143
# 00104, T 191.8, Ltr 0.4965, Lge 0.5122, Accuracy 0.8042, Recall 0.2002
# 00118, T 212.3, Ltr 0.4965, Lge 0.5194, Accuracy 0.8117, Recall 0.2171
# 00132, T 232.2, Ltr 0.5041, Lge 0.5337, Accuracy 0.8201, Recall 0.2896
# 00146, T 252.2, Ltr 0.5013, Lge 0.5338, Accuracy 0.8260, Recall 0.2709
# 00160, T 273.6, Ltr 0.5007, Lge 0.4995, Accuracy 0.8578, Recall 0.2207
# 00174, T 293.7, Ltr 0.5322, Lge 0.5083, Accuracy 0.8453, Recall 0.2218
# 00185, T 314.5, Ltr 0.4681, Lge 0.4768, Accuracy 0.8492, Recall 0.1950
# 00199, T 334.8, Ltr 0.5020, Lge 0.5120, Accuracy 0.

KeyboardInterrupt: 

In [30]:
output = train_values['outputs'][-1]
odd = utils_np.graphs_tuple_to_data_dicts(output)[0]

tdd = utils_np.graphs_tuple_to_data_dicts(train_values['target'])[0]

tdd['edges'] > 0.5

array([[ True],
       [False],
       [False],
       ...,
       [ True],
       [ True],
       [ True]])

In [33]:
odd['edges']

array([[-1.34815952],
       [-1.27758257],
       [-1.30860042],
       ...,
       [-0.52870812],
       [-0.24441601],
       [-0.07864238]])

In [42]:
odd[0]['edges'].shape

(9598, 1)

In [44]:
print(train_values['target'].edges.shape)
print(output.edges.shape)

(16998, 1)
(16998, 1)


(9598, 1)

In [46]:
tdd[0]['edges']

array([[1.],
       [0.],
       [0.],
       ...,
       [1.],
       [0.],
       [1.]])

In [47]:
odd[0]['edges']

array([[-5.57434502],
       [-5.57434871],
       [-5.5743279 ],
       ...,
       [-5.56945051],
       [-5.57278358],
       [-5.57282286]])

In [33]:
print(used_nodes <= orig_nodes)
print(orig_nodes.difference(used_nodes))
print(len(used_nodes), len(graph.nodes()))

True
{1413, 777, 1034, 781, 909, 1176, 538, 410, 923, 1150, 414, 1306, 800, 935, 40, 173, 1581, 943, 1327, 1456, 182, 567, 1336, 1213, 1471, 1472, 1476, 78, 1486, 1106, 1619, 598, 343, 217, 227, 744, 1000, 749, 1649, 885, 506, 254}
1634 1634


In [46]:
print(G.Ri[343].nonzero())
print(G.Ro[343].nonzero())

(array([], dtype=int64),)
(array([], dtype=int64),)


In [4]:
from datasets.graph import graph_to_sparse

In [10]:
S = graph_to_sparse(G)

NameError: name 'Ri' is not defined

In [8]:
print(S['Ri_rows'], S['Ri_cols'])

[   1    1    1 ... 1675 1675 1675] [   0  123  135 ... 2744 2872 2893]


In [9]:
print(S['Ro_rows'], S['Ro_cols'])

[   0    0    0 ... 1675 1675 1675] [   0    1    2 ... 3545 3546 3547]


In [13]:
print(G.Ri[0].nonzero())
print(G.Ro[0].nonzero())

(array([], dtype=int64),)
(array([0, 1, 2, 3, 4, 5, 6, 7, 8]),)


In [14]:
G.Ro[:, 0].nonzero()

(array([0]),)

In [15]:
G.Ri[:, 0].nonzero()

(array([1]),)

### Filter some hits and build graph

In [12]:
from prepare import select_hits
from prepare import split_detector_sections

In [13]:
evtid = 1
pt_min = 1
phi_slope_max = 0.001
z0_max = 200
n_phi_sections = 4
n_eta_sections = 1
filtered_hits = select_hits(hits, truth, particles, pt_min=pt_min).assign(evtid=evtid)

In [16]:
phi_range = (-np.pi, np.pi)
eta_range = [-5, 5]
phi_edges = np.linspace(*phi_range, num=n_phi_sections+1)
eta_edges = np.linspace(*eta_range, num=n_eta_sections+1)
hits_sections = split_detector_sections(filtered_hits, phi_edges, eta_edges)

In [17]:
node_feature_names = ['r', 'phi', 'z']
node_feature_scale = np.array([1000., np.pi / n_phi_sections, 1000.])

In [18]:
# Define adjacent layers
n_det_layers = 10
l = np.arange(n_det_layers)
layer_pairs = np.stack([l[:-1], l[1:]], axis=1)

In [19]:
# work on one section now
isection = 0
section_hits = hits_sections[isection]